# 04_temp_ndvi_by_mun (Temperature anomaly, temperature slope and NDVI slope)

This notebook explores the __data produced on notebook 03__ by municipality for each selected metropolis.

## Import libraries

In [1]:
from pathlib import Path

current_path = Path().resolve()

for parent in current_path.parents:
    if parent.name == "accesibilidad-urbana":
        project_root = parent
        break

print(project_root)

/home/jovyan/accesibilidad-urbana


In [2]:
import os
import sys

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

# Correlation calc
import scipy.stats as stats

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join(project_root))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup
else:
    import aup

## Notebook config

In [3]:
city = 'Guadalajara'
res = '11'

## Input

In [4]:
# Hexs with temperature data (Have no mun data)
temperature_gdf = gpd.read_file(str(project_root)+f"/data/external/temporal_fromjupyter/2025_temp_publication/temperature/temperature_res{res}_{city}.gpkg")

# Show
print(temperature_gdf.shape)
temperature_gdf.head(2)

(271809, 19)


,hex_id,res,temperature_mean,temperature_std,temperature_median,temperature_max,temperature_min,temperature_diff,temperature_tend,temperature_2018,temperature_2019,temperature_2020,temperature_2021,temperature_2022,temperature_2023,city,temperature_anomaly,temperature_slope,geometry
0,8b498c96e02efff,11,39.243659,7.477837,39.500000,48.708333,28.229167,20.479167,0.036859,36.952381,40.166667,39.973958,37.100,40.266667,39.795455,Guadalajara,1.864565,0.332612,"POLYGON ((2359127.165 962922.911, 2359145.480 ..."
1,8b498c960081fff,11,38.559783,6.846012,39.104167,47.597222,28.208333,19.388889,0.046296,36.065476,39.133333,39.020833,37.325,39.420833,39.329545,Guadalajara,1.180688,0.442486,"POLYGON ((2361405.587 958993.788, 2361423.901 ..."


In [5]:
ndvi_gdf = gpd.read_file(str(project_root)+f"/data/external/temporal_fromjupyter/2025_temp_publication/ndvi/ndvi_res{res}_{city}.gpkg")

# Show
print(ndvi_gdf.shape)
ndvi_gdf.head(2)

(271809, 19)


,hex_id,res,ndvi_mean,ndvi_std,ndvi_median,ndvi_max,ndvi_min,ndvi_diff,ndvi_tend,ndvi_2018,ndvi_2019,ndvi_2020,ndvi_2021,ndvi_2022,ndvi_2023,city,ndvi_slope,ndvi_median_rango,geometry
0,8b498c941c48fff,11,0.092168,0.028459,0.094792,0.120008,0.065130,0.054878,-0.000874,0.106006,0.105894,0.113926,0.106742,0.062115,0.058324,Guadalajara,-0.010769,Suelo artificial/Agua/Piedra,"POLYGON ((2362104.232 966205.982, 2362122.546 ..."
1,8b498c9616aafff,11,0.044711,0.015207,0.042666,0.063295,0.031247,0.032048,-0.000554,0.055969,0.062367,0.048223,0.039105,0.031626,0.030977,Guadalajara,-0.006466,Suelo artificial/Agua/Piedra,"POLYGON ((2359630.413 961412.524, 2359648.728 ..."


In [6]:
# Hexs from database (have mun data)
hexs_schema = 'hexgrid'
hexs_table = f'hexgrid_{res}_city_2020'
query = f"SELECT * FROM {hexs_schema}.{hexs_table} WHERE \"city\" = '{city}\'"
hexs_gdf = aup.gdf_from_query(query, geometry_col='geometry')

# Show
print(hexs_gdf.shape)
hexs_gdf.head(2)

(976710, 6)


,hex_id_11,geometry,CVEGEO,NOMGEO,city,type
0,8b49aa2d6809fff,"POLYGON ((-103.24414 20.50688, -103.24397 20.5...",14070,El Salto,Guadalajara,rural
1,8b498c931b2afff,"POLYGON ((-103.15693 20.54987, -103.15676 20.5...",14051,Juanacatlán,Guadalajara,rural


## Merge

In [7]:
temp_mun_gdf = pd.merge(temperature_gdf,hexs_gdf[['hex_id_11','NOMGEO']], left_on='hex_id', right_on='hex_id_11')
temp_mun_gdf.drop(columns=['hex_id_11'],inplace=True)
temp_mun_gdf.rename(columns={'NOMGEO':'nomgeo'},inplace=True)

# Show
print(temp_mun_gdf.shape)
temp_mun_gdf.head(2)

(271809, 20)


,hex_id,res,temperature_mean,temperature_std,temperature_median,temperature_max,temperature_min,temperature_diff,temperature_tend,temperature_2018,temperature_2019,temperature_2020,temperature_2021,temperature_2022,temperature_2023,city,temperature_anomaly,temperature_slope,geometry,nomgeo
0,8b498c96e02efff,11,39.243659,7.477837,39.500000,48.708333,28.229167,20.479167,0.036859,36.952381,40.166667,39.973958,37.100,40.266667,39.795455,Guadalajara,1.864565,0.332612,"POLYGON ((2359127.165 962922.911, 2359145.480 ...",Guadalajara
1,8b498c960081fff,11,38.559783,6.846012,39.104167,47.597222,28.208333,19.388889,0.046296,36.065476,39.133333,39.020833,37.325,39.420833,39.329545,Guadalajara,1.180688,0.442486,"POLYGON ((2361405.587 958993.788, 2361423.901 ...",Guadalajara


In [8]:
data_mun_gdf = pd.merge(ndvi_gdf,temp_mun_gdf, on='hex_id')

# Show
print(data_mun_gdf.shape)
data_mun_gdf.head(2)

(271809, 38)


,hex_id,res_x,ndvi_mean,ndvi_std,ndvi_median,ndvi_max,ndvi_min,ndvi_diff,ndvi_tend,ndvi_2018,...,temperature_2019,temperature_2020,temperature_2021,temperature_2022,temperature_2023,city_y,temperature_anomaly,temperature_slope,geometry_y,nomgeo
0,8b498c941c48fff,11,0.092168,0.028459,0.094792,0.120008,0.065130,0.054878,-0.000874,0.106006,...,39.931250,40.328125,37.600000,40.309375,40.355114,Guadalajara,2.078106,0.459276,"POLYGON ((2362104.232 966205.982, 2362122.546 ...",Guadalajara
1,8b498c9616aafff,11,0.044711,0.015207,0.042666,0.063295,0.031247,0.032048,-0.000554,0.055969,...,39.921429,39.205357,37.135714,39.560714,39.227273,Guadalajara,1.426806,0.227402,"POLYGON ((2359630.413 961412.524, 2359648.728 ...",Guadalajara


## Analyse by municipality

In [9]:
grouped_by = data_mun_gdf.groupby('nomgeo').agg({'temperature_mean':np.mean,
                                                 'temperature_anomaly':np.mean,
                                                 'temperature_slope':np.mean,
                                                 'ndvi_mean':np.mean,
                                                 'ndvi_slope':np.mean})

# Show
grouped_by

,temperature_mean,temperature_anomaly,temperature_slope,ndvi_mean,ndvi_slope
nomgeo,,,,,
El Salto,37.955879,0.576784,0.324694,0.229695,-0.026244
Guadalajara,37.907774,0.528679,0.318289,0.149576,-0.015275
Juanacatlán,37.307453,-0.071641,0.374280,0.269941,-0.032004
San Pedro Tlaquepaque,37.796279,0.417185,0.343766,0.194293,-0.021722
Tlajomulco de Zúñiga,37.554486,0.175391,0.144324,0.249002,-0.023722
Tonalá,37.460956,0.081861,0.298470,0.222286,-0.025449
Zapopan,36.657073,-0.722021,0.242723,0.221813,-0.022091
